In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd 
import os
from tensorflow.keras.initializers import RandomNormal
import plotly
import plotly.graph_objects as go

In [9]:
def save_to_files(model):
    multiplyer = 10

    os.remove("bias.dat")
    f = open("bias.dat", "a")
    for layer in model.layers:
      for bias in layer.get_weights()[1]:
        f.write(str(round(bias*multiplyer))+",\n")
    f.close()

    os.remove("weights.dat")
    f = open("weights.dat", "a")
    for layer in model.layers:
      for neuron in layer.get_weights()[0]:
        for weight in neuron:
          f.write(str(round(weight*multiplyer))+",\n")
    f.close()
    
def transform_and_save_new_static_files(input_shape, reshaped_training_data, reshaped_test_data):
    pca = PCA(n_components=input_shape)
    X_pca_train = pca.fit_transform(reshaped_training_data)
    X_pca_test = pca.transform(reshaped_test_data)
    #Save to static data
    pd.DataFrame(X_pca_train).to_pickle("./pca_applied_training_images.pkl")
    pd.DataFrame(X_pca_test).to_pickle("./pca_applied_test_images.pkl")

def plot_accuracy_vs(x,y,z):
    fig = go.Figure(data=[
        go.Bar(name='pre_rounded_results', x=x, y=y),
        go.Bar(name='post_rounded_results', x=x, y=z)    
    ])

    fig.update_layout(
        barmode='group',
        title="Accuracy comparison across architectures and neural network weight rounding",
        xaxis_title="Arhitecture (u=units/neurons, L=number of layers)",
        yaxis_title="Accuracy",
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="Black"
        )
    )
    fig.show()
def generate_pre_and_post(model, 
                          X_pca_test: np.ndarray, 
                          y_test: np.ndarray, 
                          neurons: int, 
                          weight_rounding: int, 
                          image_rounding: int, 
                          no_of_layers: int, 
                          x: list, 
                          y: list, 
                          z: list) -> list:
    weights = []
    oldWeights = model.get_weights()
    pre_rounded_results = model.evaluate(X_pca_test.round(image_rounding), 
                                         y_test, 
                                         verbose=0)
    for layer in model.get_weights():
        weights.append(layer.round(weight_rounding))
    model.set_weights(weights) 
    post_rounded_results = model.evaluate(X_pca_test.round(image_rounding), 
                                          y_test, 
                                          verbose=0)
    print("Error of rounding weights to {}-decimal: {}".format(weight_rounding, pre_rounded_results[1]-post_rounded_results[1]))
    x.append("{}L, {}u".format(no_of_layers,neurons))
    y.append(pre_rounded_results[1])
    z.append(post_rounded_results[1])
    #df = pd.concat([pd.Series(x),pd.Series(y), pd.Series(z)], axis=1).rename(columns={0:"network_architecture", 
    #                                                                                  1:"pre_rounded_results",
    #                                                                                  2:"post_rounded_results"})
    return [x,y,z]

def fit_and_evaluate_model(X_pca_train: np.ndarray, 
                           y_train: np.ndarray, 
                           batch_size: int, 
                           epochs: int, 
                           neurons: int, 
                           num_classes: int, 
                           no_of_layers: int):
    NN_layers =  []
    NN_layers.append(keras.Input(shape=input_shape))
    for i in range(no_of_layers):
        NN_layers.append(layers.Dense(neurons, activation="relu"))
    NN_layers.append(layers.Dense(num_classes, activation="softmax"))
    
    model = keras.Sequential(NN_layers)
    model.compile(loss="categorical_crossentropy", 
                  optimizer="adam", 
                  metrics=["accuracy"])
    model.fit(X_pca_train, 
              y_train, 
              batch_size=batch_size, 
              epochs=epochs,
              verbose=1)
    return model

## Prepare the data

In [10]:
# Model / data parameters
num_classes = 10
input_shape = 80

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

reshaped_training_data = x_train.ravel().reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
reshaped_test_data = x_test.ravel().reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])

print("x_train shape:", reshaped_training_data.shape)
print(reshaped_training_data.shape[0], "train samples")
print(reshaped_test_data.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 784)
60000 train samples
10000 test samples


## Build the model

In [11]:
#Import static variables. 
model = tf.keras.models.load_model("./current_model")
X_pca_train = pd.read_pickle("./pca_applied_training_images.pkl").values
X_pca_test = pd.read_pickle("./pca_applied_test_images.pkl").values

In [58]:
x = []
y = []
z = []
#Single model
[x,y,z] = generate_pre_and_post(model=model,
                                X_pca_test=X_pca_test,
                                y_test=y_test, 
                                neurons=70, 
                                weight_rounding=1, 
                                image_rounding=0,
                                no_of_layers=2,
                                x=x, 
                                y=y, 
                                z=z)
plot_accuracy_vs(x,y,z)

Error of rounding to 1-decimal: 0.0


## Multiple model head-to-head

In [68]:
batch_size = 128
epochs = 30
units = [64,128,256,512]#[10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250,260,270,280,290,300]
no_of_layers = [2,3,4]
weight_rounding=1
image_rounding=0
x = []
y = []
z = []
#Multi model setup
for number_of_layers in no_of_layers:
    for i in units:
        print(i*10)
        model = fit_and_evaluate_model(X_pca_train=X_pca_train, 
                                       y_train=y_train, 
                                       batch_size=batch_size, 
                                       epochs=epochs, 
                                       neurons=i,
                                       num_classes=num_classes,
                                       no_of_layers=number_of_layers)
        [x,y,z] = generate_pre_and_post(model= model,
                                        X_pca_test= X_pca_test,
                                        y_test=y_test, 
                                        neurons=i, 
                                        weight_rounding=weight_rounding, 
                                        image_rounding=image_rounding,
                                        no_of_layers=number_of_layers,
                                        x=x, 
                                        y=y, 
                                        z=z)
plot_accuracy_vs(x,y,z)


640
Epoch 1/30
469/469 [==============================] - 2s 3ms/step - loss: 0.4977 - accuracy: 0.8596
Epoch 2/30
469/469 [==============================] - 1s 3ms/step - loss: 0.1763 - accuracy: 0.9482
Epoch 3/30
469/469 [==============================] - 1s 3ms/step - loss: 0.1314 - accuracy: 0.9625: 0s - loss: 0.136
Epoch 4/30
469/469 [==============================] - 1s 3ms/step - loss: 0.1056 - accuracy: 0.9681
Epoch 5/30
469/469 [==============================] - 2s 3ms/step - loss: 0.0884 - accuracy: 0.9741
Epoch 6/30
469/469 [==============================] - 1s 3ms/step - loss: 0.0758 - accuracy: 0.9778
Epoch 7/30
469/469 [==============================] - 1s 3ms/step - loss: 0.0665 - accuracy: 0.9808
Epoch 8/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0588 - accuracy: 0.9825
Epoch 9/30
469/469 [==============================] - 1s 3ms/step - loss: 0.0523 - accuracy: 0.9846
Epoch 10/30
469/469 [==============================] - 1s 3ms/step - loss: 0.0

469/469 [==============================] - 2s 3ms/step - loss: 0.3285 - accuracy: 0.9052
Epoch 2/30
469/469 [==============================] - 2s 4ms/step - loss: 0.1056 - accuracy: 0.9686
Epoch 3/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0726 - accuracy: 0.9782: 0s - loss: 0.0716 - accuracy
Epoch 4/30
469/469 [==============================] - 2s 5ms/step - loss: 0.0546 - accuracy: 0.9830: 0s - l
Epoch 5/30
469/469 [==============================] - 2s 5ms/step - loss: 0.0406 - accuracy: 0.9876
Epoch 6/30
469/469 [==============================] - 2s 5ms/step - loss: 0.0322 - accuracy: 0.9898
Epoch 7/30
469/469 [==============================] - 2s 5ms/step - loss: 0.0248 - accuracy: 0.9919
Epoch 8/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0218 - accuracy: 0.9930
Epoch 9/30
469/469 [==============================] - 2s 4ms/step - loss: 0.0165 - accuracy: 0.9949
Epoch 10/30
469/469 [==============================] - 2s 4ms/step - loss

469/469 [==============================] - 3s 6ms/step - loss: 0.0323 - accuracy: 0.9893
Epoch 6/30
469/469 [==============================] - 3s 7ms/step - loss: 0.0260 - accuracy: 0.9922
Epoch 7/30
469/469 [==============================] - 3s 6ms/step - loss: 0.0231 - accuracy: 0.9923
Epoch 8/30
469/469 [==============================] - 3s 6ms/step - loss: 0.0222 - accuracy: 0.9927
Epoch 9/30
469/469 [==============================] - 3s 7ms/step - loss: 0.0186 - accuracy: 0.9937
Epoch 10/30
469/469 [==============================] - 3s 6ms/step - loss: 0.0164 - accuracy: 0.9944
Epoch 11/30
469/469 [==============================] - 3s 6ms/step - loss: 0.0173 - accuracy: 0.9944: 0s - loss: 0.0175 - accura
Epoch 12/30
469/469 [==============================] - 3s 6ms/step - loss: 0.0131 - accuracy: 0.9959
Epoch 13/30
469/469 [==============================] - 3s 7ms/step - loss: 0.0097 - accuracy: 0.9970
Epoch 14/30
469/469 [==============================] - 3s 7ms/step - loss: 0.01

In [ ]:
#Architecture documentation
plot_accuracy_vs(x,y,z)
df = pd.DataFrame(a)
df[(df.max(axis=1) > 0.4) & (df.max(axis=1) < 0.6)].max(axis=1)

In [ ]:
x = ["2l,128u", "3l,128u", "2l,256u", "3l,256u", "2l,512u", "3l,512u"]
y = [0.9617000222206116, 0.9593999981880188, 0.9624000191688538, 0.9639000296592712, 0.9681000113487244, 0.96670001745224]
z = [0.9556000232696533, 0.954200029373169, 0.9595000147819519, 0.9596999883651733, 0.9621000289916992, 0.9645000100135803]

In [7]:
#Direct evaluation
import random
current_image = 210#random.randint(0, len(x_test))
plt.imshow(x_test[current_image].reshape(28,28))
plt.show()
print(y_test[current_image])
input_image = (X_pca_test[current_image]).round(0).astype(int)
a = model.predict(X_pca_test.round(0))
n=0
for i in a[current_image]:
    print("certancy of class {}: {:,.7f}".format(n, i))
    n+=1
input_image

NameError: name 'x_test' is not defined

In [6]:
from math import exp
current_image = 210
def execute_relu(neuron_vector: list) -> pd.Series:
    n = pd.Series(neuron_vector)
    n[n < 0] = 0
    return n

def execute_layer(Weight_matrix: pd.DataFrame, input_neurons: pd.Series) -> pd.Series:
    q=[]
    for i in range(Weight_matrix.shape[1]):
        q.append(sum(Weight_matrix[i]*input_neurons))
    layer_vector = execute_relu(q)
    return pd.Series(q).append(pd.Series(1)).reset_index(drop=True) #Added the one for the next bias
def execute_softmax(just_before_output):
    exps= []
    for i in just_before_output:
        exps.append(exp(i))
    q = pd.Series(exps)

    p0 = q[0] / q.sum()
    p1 = q[1] / q.sum()
    p2 = q[2] / q.sum()
    p3 = q[3] / q.sum()
    p4 = q[4] / q.sum()
    p5 = q[5] / q.sum()
    p6 = q[6] / q.sum()
    p7 = q[7] / q.sum()
    p8 = q[8] / q.sum()
    p9 = q[9] / q.sum()
    total =[p0,p1,p2,p3,p4,p5,p6,p7,p8,p9]
    n = 0
    for i in total:
        print("Certancy of class {}: {:,.7f}".format(n, i))
        n+=1

W1 = pd.DataFrame(model.layers[0].get_weights()[0]).append(pd.DataFrame(model.layers[0].get_weights()[1]).T).reset_index(drop=True)
W2 = pd.DataFrame(model.layers[1].get_weights()[0]).append(pd.DataFrame(model.layers[1].get_weights()[1]).T).reset_index(drop=True)
W3 = pd.DataFrame(model.layers[2].get_weights()[0]).append(pd.DataFrame(model.layers[2].get_weights()[1]).T).reset_index(drop=True)
input_img = pd.Series(X_pca_test.round(0)[current_image]).append(pd.Series(1)).reset_index(drop=True)

neurons_h1 = execute_layer(W1, input_img)
neurons_h2 = execute_layer(W2, neurons_h1)
just_before_output = execute_layer(W3, neurons_h2)

execute_softmax(just_before_output=just_before_output)

Certancy of class 0: 0.0000000
Certancy of class 1: 0.0000000
Certancy of class 2: 0.0000000
Certancy of class 3: 0.0000000
Certancy of class 4: 0.0392410
Certancy of class 5: 0.0000000
Certancy of class 6: 0.9607590
Certancy of class 7: 0.0000000
Certancy of class 8: 0.0000000
Certancy of class 9: 0.0000000
